In [19]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
%cd '/content/drive/My Drive/Colab Notebooks/2-parametro-ventas-insumos'

/content/drive/My Drive/Colab Notebooks/2-parametro-ventas-insumos


In [22]:
import os
import random as rd
import numpy as np
import pandas as pd

In [23]:
ruta_iniciales = os.path.join(os.getcwd(), 'archivos-iniciales')
ruta_resultados = os.path.join(os.getcwd(), 'archivos-resultados')

In [24]:
archivos_iniciales = os.listdir(ruta_iniciales)
archivos_iniciales

['reps-ids-cacao.csv',
 'reps-ids-maiz.csv',
 'reps-ids-arroz.csv',
 'arroz.csv',
 'maiz.csv',
 'cacao.csv']

IDS UNICOS

In [25]:
maiz = pd.read_csv(os.path.join(ruta_iniciales, 'maiz.csv'))

IDS REPETIDOS

In [26]:
maiz_reps = pd.read_csv(os.path.join(ruta_iniciales, 'reps-ids-maiz.csv'))

<hr>

transitorios:
  > [arroz, maiz]

permanentes
  > [cacao]

'Ventas' para arroz y maiz

'ventas' para cacao

### CORRIGIENDO LAS UNIDADES

FUNCIONES

In [30]:
def conversion_tonelada_litro(df, columna_unidades, columna_valor):
  mascara = df[columna_unidades] == 4
  df[columna_valor][mascara] = df[columna_valor][mascara] * 2831.6846592
  return df  

def conversion_quintal_litro(df, columna_unidades, columna_valor):
  mascara = df[columna_unidades] == 3
  df[columna_valor][mascara] = df[columna_valor][mascara] * 100 * 0.45359236996888618876
  return df

def conversion_kilogramo_litro(df, columna_unidades, columna_valor):
  mascara = df[columna_unidades] == 2
  df[columna_valor][mascara] = df[columna_valor][mascara] * 1
  return df

def conversion_libra_litro(df, columna_unidades, columna_valor):
  mascara = df[columna_unidades] == 1
  df[columna_valor][mascara] = df[columna_valor][mascara] * 0.45359236996888618876
  return df

In [31]:
maiz_reps = conversion_tonelada_litro(maiz_reps, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')
maiz_reps = conversion_quintal_litro(maiz_reps, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')
maiz_reps = conversion_kilogramo_litro(maiz_reps, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')
maiz_reps = conversion_libra_litro(maiz_reps, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')

maiz = conversion_tonelada_litro(maiz, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')
maiz = conversion_quintal_litro(maiz, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')
maiz = conversion_kilogramo_litro(maiz, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')
maiz = conversion_libra_litro(maiz, 'ct_umed_npk_fq', 'ct_cantidad_npk_fq')

### FUNCIONES

In [32]:
def parametro_ventas(df_repetidos, df_unicos, col_superficie, col_ventas):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  df_unicos['parametros_ventas'] = 0
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    
    array_superficie_cos = df_repetidos[mascara][col_superficie].values
    array_superficie_cos = np.nan_to_num(array_superficie_cos, nan=0)
    superficie_cosechada = array_superficie_cos.sum()
    
    array_ventas = df_repetidos[mascara][col_ventas].values
    array_ventas = np.nan_to_num(array_ventas, nan=0)
    ventas = array_ventas.sum()

    division = ventas / superficie_cosechada
    df_unicos.loc[id, 'parametro_ventas'] = division
  
  df_unicos.reset_index(inplace=True)
  return df_unicos


def parametro_insumos(df_repetidos, df_unicos, col_superficie, col_insumos):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  df_unicos['parametros_ventas'] = 0
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    
    array_superficie_plt = df_repetidos[mascara][col_superficie].values
    array_superficie_plt = np.nan_to_num(array_superficie_plt, nan=0)
    superficie_plantada = array_superficie_plt.sum()
    
    array_insumos = df_repetidos[mascara][col_insumos].values
    array_insumos = np.nan_to_num(array_insumos, nan=0)
    insumos = array_insumos.sum()

    division = insumos / superficie_plantada
    df_unicos.loc[id, 'parametro_insumos'] = division
  
  df_unicos.reset_index(inplace=True)
  return df_unicos

### maiz

In [33]:
# muestra = rd.sample(list(maiz_reps['Identificador'][maiz_reps['Identificador'].duplicated()]), 5)
muestra = [23015001070720001, 13155002187840005, 12045102077630006, 1075001206870001, 13115002198790001]
muestra

[23015001070720001,
 13155002187840005,
 12045102077630006,
 1075001206870001,
 13115002198790001]

In [34]:
maiz_reps[maiz_reps['Identificador'].isin(muestra)][['Identificador', 'ct_k511', 'ct_k509', 'Ventas', 'ct_cantidad_npk_fq']]

,Identificador,ct_k511,ct_k509,Ventas,ct_cantidad_npk_fq
206,1075001206870001,0.50,7.0,0.272727,NaN
207,1075001206870001,0.50,12.0,0.272727,NaN
936,13115002198790001,2.00,9.0,4.545455,NaN
937,13115002198790001,6.00,4.0,22.727273,NaN
1108,13155002187840005,5.00,6.0,11.363636,1500.0
1109,13155002187840005,6.50,6.0,14.772727,1950.0
1110,13155002187840005,0.50,6.0,1.136364,150.0
1289,23015001070720001,3.00,12.0,6.236364,150.0
1290,23015001070720001,3.00,8.0,6.109091,150.0
1291,23015001070720001,3.00,4.0,6.236364,150.0


In [35]:
maiz = parametro_ventas(maiz_reps, maiz, 'ct_k511', 'Ventas')
maiz = parametro_insumos(maiz_reps, maiz, 'ct_k509', 'ct_cantidad_npk_fq')

In [36]:
maiz[maiz['Identificador'].isin(muestra)][['Identificador', 'ct_k511', 'ct_k509', 'Ventas', 'parametro_insumos', 'parametro_ventas']]

,Identificador,ct_k511,ct_k509,Ventas,parametro_insumos,parametro_ventas
162,1075001206870001,0.5,7.0,0.545455,0.000000,0.545455
750,13115002198790001,2.0,9.0,27.272727,0.000000,3.409091
882,13155002187840005,5.0,6.0,27.272727,200.000000,2.272727
1025,23015001070720001,3.0,12.0,18.581818,18.750000,2.064646
1213,12045102077630006,6.0,6.0,43.636364,166.666667,3.642434


In [37]:
maiz['parametro_insumos'].replace([np.inf, -np.inf], np.nan, inplace=True)
maiz['parametro_ventas'].replace([np.inf, -np.inf], np.nan, inplace=True)
maiz[maiz['Identificador'].isin(muestra)][['Identificador', 'ct_k511', 'ct_k509', 'Ventas', 'parametro_insumos', 'parametro_ventas']]

,Identificador,ct_k511,ct_k509,Ventas,parametro_insumos,parametro_ventas
162,1075001206870001,0.5,7.0,0.545455,0.000000,0.545455
750,13115002198790001,2.0,9.0,27.272727,0.000000,3.409091
882,13155002187840005,5.0,6.0,27.272727,200.000000,2.272727
1025,23015001070720001,3.0,12.0,18.581818,18.750000,2.064646
1213,12045102077630006,6.0,6.0,43.636364,166.666667,3.642434


### GUARDADO

In [38]:
maiz.to_csv(os.path.join(ruta_resultados, 'maiz.csv'), index=False, encoding='utf-8')

In [39]:
maiz.to_stata(os.path.join(ruta_resultados, 'maiz.dta'), write_index=False)